<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/RAG_with_Llama_3_70b_in_Langchain_Chat_with_PDF_using_Free_Embeddings%2C_Reranker_%26_LlamaParse_31_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1 --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.4 requires langchain-core<0.4.0,>=0.3.12, but you have langchain-core 0.1.52 which is incompatible.
langchain-text-splitters 0.3.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.1.52 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.3 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.3 which is incompatible.
tensorboard 2.17.0 require

In [25]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

In [5]:
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [6]:
data =  "/content/Meta-06-30-2024-Exhibit-99-1_Final.pdf"

In [7]:
#Document parsing

instruction = """The provided document is Meta First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key = userdata.get("LLAMA_PARSE"),
    result_type = "markdown",
    parsing_instruction = instruction,
    max_timeout = 5000
)

llama_parse_documents = await parser.aload_data(data)

Started parsing the file under job_id 86549ac2-862f-4407-8e0b-51dc02d65959


In [8]:
parsed_doc = llama_parse_documents[0]

In [9]:
Markdown(parsed_doc.text[:4096])

# Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the year," said Mark Zuckerberg, Meta founder and CEO. "We've released the first frontier-level open source AI model, we continue to see good traction with our Ray-Ban Meta AI glasses, and we're driving good growth across our apps."

# Second Quarter 2024 Financial Highlights

|In millions, except percentages and per share amounts|2024|2023|% Change|
|---|---|---|---|
|Revenue|$39,071|$31,999|22 %|
|Costs and expenses|$24,224|$22,607|7 %|
|Income from operations|$14,847|$9,392|58 %|
|Operating margin|38 %|29 %| |
|Provision for income taxes|$1,641|$1,505|9 %|
|Effective tax rate|11 %|16 %| |
|Net income|$13,465|$7,788|73 %|
|Diluted earnings per share (EPS)|$5.16|$2.98|73 %|

# Second Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.27 billion on average for June 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 10% year-over-year.
- Average price per ad – Average price per ad increased by 10% year-over-year.
- Revenue – Total revenue was $39.07 billion, an increase of 22% year-over-year. Revenue on a constant currency basis would have increased 23% year-over-year.
- Costs and expenses – Total costs and expenses were $24.22 billion, an increase of 7% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $8.47 billion.
- Capital return program – Share repurchases were $6.32 billion of our Class A common stock and dividend payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.08 billion as of June 30, 2024. Free cash flow was $10.90 billion.
- Headcount – Headcount was 70,799 as of June 30, 2024, a decrease of 1% year-over-year.
---
# CFO Outlook Commentary

We expect third quarter 2024 total revenue to be in the range of $38.5-41 billion. Our guidance assumes foreign currency is a 2% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, unchanged from our prior outlook. For Reality Labs, we continue to expect 2024 operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and investments to further scale our ecosystem.

While we do not intend to provide any quantitative guidance for 2025 until the fourth quarter call, we expect infrastructure costs will be a significant driver of expense growth next year as we recognize depreciation and operating costs associated with our expanded infrastructure footprint.

We anticipate our full-year 2024 capital expenditures will be in the range of $37-40 billion, updated from our prior range of $35-40 billion. While we continue to refine our plans for next year, we currently expect significant capital expenditures growth in 2025 as we invest to support our artificial intelligence research and product development efforts.

Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory headwinds in the EU and the U.S. that could significantly impact our business and our financial results.
---
# Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at the Meta Investor Relations website at investor.fb.com, along with the earnings press release, financial tables, and slide presentation.

Following the call, a replay will be available at the same website. Transcripts of conferen

In [10]:
document_path = Path("parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [11]:
# Vector Embeddings
loader = UnstructuredMarkdownLoader(document_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(documents)
len(docs)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


12

In [12]:
print(docs[0].page_content)


Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the year," said Mark Zuckerberg, Meta founder and CEO. "We've released the first frontier-level open source AI model, we continue to see good traction with our Ray-Ban Meta AI glasses, and we're driving good growth across our apps."

Second Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts 2024 2023 % Change Revenue $39,071 $31,999 22 % Costs and expenses $24,224 $22,607 7 % Income from operations $14,847 $9,392 58 % Operating margin 38 % 29 % Provision for income taxes $1,641 $1,505 9 % Effective tax rate 11 % 16 % Net income $13,465 $7,788 73 % Diluted earnings per share (EPS) $5.16 $2.98 73 %

Second Quarter 2024 Operational and Other Financial Highlight

In [13]:
embeddings = FastEmbedEmbeddings(model_name= "BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [14]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path = "./db",
    collection_name = "document_embeddings"
)

In [15]:
%%time
query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 444 ms, sys: 3.53 ms, total: 447 ms
Wall time: 467 ms


In [16]:
for doc,score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used 

score: 0.6170933148970821
--------------------------------------------------------------------------------

text: Following the call, a replay will be available at the same website. Transcripts of conference calls with publishing equity research analysts held today will also be posted to the investor.fb.com website.

Disclosure Information

Meta uses the investor.fb.c

score: 0.5631786671492258
--------------------------------------------------------------------------------

text: Headcount – Headcount was 70,799 as of June 30, 2024, a decrease of 1% year-over-year.

CFO Outlook Commentary

We expect third quarter 2024 total revenue to be in the range of $38.5-41 billion. Our guidance assumes foreign currency is a 2% headwind 

In [17]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k":5})
retrieved_docs = retriever.invoke(query)

CPU times: user 290 ms, sys: 3.41 ms, total: 294 ms
Wall time: 294 ms


In [18]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()


id: e6bfdf3fc6674c87b1a936ec822d2753

text: Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used 

--------------------------------------------------------------------------------

id: 00f2276161db44798e4677fc22425f6a

text: Following the call, a replay will be available at the same website. Transcripts of conference calls with publishing equity research analysts held today will also be posted to the investor.fb.com website.

Disclosure Information

Meta uses the investor.fb.c

--------------------------------------------------------------------------------

id: 5364e493a51c4631be236bd463901526

text: Headcount – Headcount was 70,799 as of June 30, 2024, a decrease of 1% year-over-year.

CFO Outlook Commentary

We expect third quarter 2024 total revenue to be in the range of $38.5-41 bi

In [20]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compressor_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 36.9MiB/s]


In [21]:
%%time
reranked_docs = compressor_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.4 s, sys: 192 ms, total: 2.59 s
Wall time: 2.49 s


3

In [22]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: e6bfdf3fc6674c87b1a936ec822d2753

text: Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used 

score: 0.50009685754776
--------------------------------------------------------------------------------

id: 00f2276161db44798e4677fc22425f6a

text: Following the call, a replay will be available at the same website. Transcripts of conference calls with publishing equity research analysts held today will also be posted to the investor.fb.com website.

Disclosure Information

Meta uses the investor.fb.c

score: 0.0143993915989995
--------------------------------------------------------------------------------

id: 5364e493a51c4631be236bd463901526

text: Headcount – Headcount was 70,799 as of June 30, 2024, a decrease of 1% year-over-year.

CFO Outlook Commentary

We expect third quarter 20

In [23]:
#Q&A Over Document

llm = ChatGroq(temperature= 0, model_name = "llama3-70b-8192")

In [24]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

In [26]:
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [27]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = compressor_retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt, "verbose": True}
)

In [28]:
%%time
response = qa.invoke("What is the most significant innovation from Meta?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif. – July 31, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the year," said Mark Zuckerberg, Meta founder and CEO. "We've released the first frontier-level open source AI model, we continue to see good traction with our Ray-Ban Meta AI glasses, and we're driving good growth across our apps."

Second Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts 2024 2023 % Change Revenue $39,071 $31,999 22 % Costs and expenses 

In [29]:
print_response(response)

Based on the provided information, the most significant innovation from Meta is Meta AI, which is on
track to be the most used AI assistant in the world by the end of the year. Additionally, Meta has
released the first frontier-level open source AI model and is seeing good traction with its Ray-Ban
Meta AI glasses.

It's worth noting that Meta is also investing heavily in artificial intelligence research and
product development efforts, with plans to increase capital expenditures significantly in 2025 to
support these initiatives.


In [30]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compressor_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)


In [31]:
%%time
response = qa.invoke("What is the revenue for 2024 and % change?")

Running pairwise ranking..
CPU times: user 2.56 s, sys: 110 ms, total: 2.67 s
Wall time: 3.14 s


In [32]:
Markdown(response["result"])

**Revenue for 2024 and % Change:**

* GAAP revenue for 2024: $75,527 million (six months ended June 30, 2024)
* GAAP revenue year-over-year change: 25%

**Additional Helpful Information:**

* Revenue excluding foreign exchange effect: $75,792 million (six months ended June 30, 2024)
* Revenue excluding foreign exchange effect year-over-year change: 25%

Note: The revenue figures are for the six months ended June 30, 2024, and the year-over-year changes are compared to the same period in 2023.

In [33]:
%%time
response = qa.invoke("What is the revenue for 2023?")

Running pairwise ranking..
CPU times: user 3.21 s, sys: 100 ms, total: 3.31 s
Wall time: 4.05 s


In [34]:
print_response(response)

**Answer:** The revenue for 2023 is not explicitly stated for the entire year. However, the revenue
for the three months ended June 30, 2023, and the six months ended June 30, 2023, are provided:

* Three months ended June 30, 2023: $31,999
* Six months ended June 30, 2023: $60,645

**Additional helpful information:**

* The year-over-year revenue growth rate is 22% for the three months ended June 30, 2024, and 25%
for the six months ended June 30, 2024.
* The revenue excluding foreign exchange effect is also provided, which shows a year-over-year
growth rate of 23% for both the three months and six months ended June 30, 2024.


In [35]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)


Running pairwise ranking..
CPU times: user 2.58 s, sys: 110 ms, total: 2.69 s
Wall time: 3.63 s


In [36]:
print_response(response)

Based on the provided information, we can calculate the revenue minus the costs and expenses for
2024 as follows:

Revenue: $75,527 (six months ended June 30, 2024)
Total Expenses: $96-99 billion (full-year 2024 guidance)

Since we don't have the exact total expenses figure, we can't provide an exact answer. However, we
can estimate the range of revenue minus costs and expenses for 2024:

Revenue minus costs and expenses = $75,527 (six months) x 2 (to annualize) = $151,054 (estimated
annual revenue)
Estimated annual revenue minus costs and expenses = $151,054 - $96,000,000,000 (lower end of expense
guidance) = -$94,948,946,000
Estimated annual revenue minus costs and expenses = $151,054 - $99,000,000,000 (upper end of expense
guidance) = -$98,848,946,000

Keep in mind that these estimates are based on the provided guidance and may not reflect the actual
figures.

Additional helpful information:

* The company expects significant capital expenditures growth in 2025 to support artificial

In [37]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)


Running pairwise ranking..
CPU times: user 2.51 s, sys: 107 ms, total: 2.61 s
Wall time: 3.15 s


In [38]:
print_response(response)

Based on the provided information, we can calculate the revenue minus costs and expenses for 2023 as
follows:

Revenue (2023) = $60,645 (from the Reconciliation of GAAP to Non-GAAP Results table)

To find the costs and expenses, we can use the Net Income figure and add back Depreciation and
Amortization, Share-based Compensation, and other adjustments. However, this information is not
explicitly provided for 2023. We can try to estimate it using the 2024 figures, but this would not
be accurate.

Unfortunately, the provided information does not allow us to accurately calculate the revenue minus
costs and expenses for 2023.


In [39]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..
CPU times: user 2.49 s, sys: 94.3 ms, total: 2.58 s
Wall time: 3.06 s


In [40]:
Markdown(response["result"])

The question asks about the expected revenue for the second quarter of 2024, but the provided information does not contain any guidance or expectations for the second quarter of 2024. The CFO Outlook Commentary provides guidance for the third quarter of 2024 and full-year 2024, but not for the second quarter of 2024.

However, the provided information does contain the actual revenue for the second quarter of 2024, which is $39.07 billion.

Here is the relevant information:

**Second Quarter 2024 Financial Highlights**

* Revenue: $39,071 million, an increase of 22% year-over-year.

In [41]:
%%time
response = qa.invoke("What is the overall outlook of Q1 2024?")

Running pairwise ranking..
CPU times: user 2.46 s, sys: 99.2 ms, total: 2.56 s
Wall time: 3.22 s


In [42]:
print_response(response)


**The question is asking about Q1 2024, but the provided information is about Q2 2024.**

Since there is no information about Q1 2024, I don't know the answer to this question.

However, I can provide some information about Q2 2024:

* Headcount decreased by 1% year-over-year to 70,799 as of June 30, 2024.
* Total revenue is expected to be in the range of $38.5-41 billion for Q3 2024.
* Full-year 2024 total expenses are expected to be in the range of $96-99 billion.
* Capital expenditures are expected to be in the range of $37-40 billion for full-year 2024.

Please note that these pieces of information are about Q2 2024 and full-year 2024, not Q1 2024.
